In [1]:
from openai import OpenAI
import json
import chromadb

# create an OpenAI client
client = OpenAI()

#create a chromaDB client
chroma_client = chromadb.PersistentClient()
collection = chroma_client.get_or_create_collection(name="Real_Estate" , metadata={"hnsw:space": "cosine"} )

# Check if there is any existing data in the collection
collection.count()


30

In [2]:
questions = [   
                "How big do you want your house to be?" ,
                "What are 3 most important things for you in choosing this property?", 
                "Which amenities would you like?", 
                "Which transportation options are important to you?",
                "How urban do you want your neighborhood to be?",   
            ]

In [3]:
answers = []
for i,question in enumerate(questions):
    answers.append(input(question + "\n"))


In [4]:
results = collection.query(
    query_texts=answers, # Chroma will embed this for you
    n_results=3 # how many results to return
)

In [5]:
House_list = [item.split("_")[0] for sublist in results["ids"] for item in sublist]

In [6]:
Unique_House_list = list(set(House_list))

In [7]:
Unique_House_list


['db5f6eca-7eea-4a7f-bfcf-aad6d5fb6c75',
 '252ea444-9531-4442-a3d0-aaa2d83d2d2c',
 '327f9aa5-d09f-45fb-b617-b8b8163506ad',
 '0d1901f4-5b46-42ea-bbd2-addc00e66b89',
 'bfb32624-d60b-4674-a8c0-fd48f003e52f',
 '51eac004-507a-42c6-8689-f11376fe6d09',
 '76e31b37-8b13-4b0c-b426-2a89e8f43dde',
 '237a41ef-a757-4428-abc4-f8fdbc1ff654']

In [8]:
print(f"Here are the {len(Unique_House_list)} houses that match your preferences:")

for i,id in enumerate(Unique_House_list):
    print(f"House {i+1}: \n")
    Des_data = collection.get(ids=[f"{id}_Description"])
    #print(Des_data)
    Description = Des_data['documents'][0]
    Nei_Des_data = collection.get(ids=[f"{id}_Neighborhood_Description"])
    Neighborhood_Description = Nei_Des_data['documents'][0]
    #print(Description,Neighborhood_Description)

    Userpreferences = " "
    for j,question in enumerate(questions):
        Userpreferences = Userpreferences + "Question:" + questions[j] + "User Preference:" + answers[j] + "\n "

    Example = "Comfortable house with a spacious kitchen and a cozy living room in the charming Ravenswood neighborhood. This area is known for its tree-lined streets, historic architecture, and welcoming community vibe. You will find good local schools, convenient shopping options, and a quiet neighborhood in Ravenswood. Additionally, with easy access to the Metra station and public transportation, commuting to downtown Chicago will be a breeze. Look for a property with a backyard for gardening, a two-car garage, and a modern, energy-efficient heating system to suit your needs"
    messages = [{"role": "user", "content": f"Given the information of House Description as {Description} and Neighborhood Description as {Neighborhood_Description} , User's preference is: {Userpreferences}  provide a personalized message for the user. Example: {Example}"}]
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
    )
    response_message = response.choices[0].message.content
    print(response_message)






Here are the 8 houses that match your preferences:
House 1: 



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Charming 3-bedroom, 2-bathroom home in the heart of Sunnyvale is the perfect place for you. This cozy house features a comfortable living room with a fireplace, a bright kitchen with stainless steel appliances, and a spacious backyard for all your outdoor activities. Located in a quiet neighborhood with top-rated schools and convenient shopping options nearby, you will feel right at home in this vibrant community. With easy access to major highways and a reliable bus line, commuting will be a breeze. Enjoy the perfect balance of suburban tranquility and access to urban amenities in Sunnyvale. Get ready to call this wonderful place home!
House 2: 

Cozy three-bedroom house with a spacious kitchen and a relaxing living room await you in the delightful Prairie Ridge neighborhood. Known for its excellent schools, parks, and recreational facilities, this vibrant community offers a perfect blend of suburban tranquility and urban convenience. Enjoy the convenience of nearby shops and restaura